# Packages used

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.io as sio
from coroica import CoroICA, UwedgeICA
from tqdm.notebook import tqdm

from scipy.signal import butter, lfilter
from sklearn.decomposition import FastICA
from sklearn.linear_model import LinearRegression

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier

## Band-Pass filter options

In [ ]:
# Band-Pass filter options
fs = 250 # sampling frequency
order = 3
nyq = 0.5 * fs
# BETA
lowcut = 13
highcut = 30
low = lowcut / nyq
high = highcut / nyq
b_beta, a_beta = butter(order, [low, high], btype='band')

#ALPHA

lowcut_alpha = 8
highcut_alpha = 13
low_alpha = lowcut_alpha / nyq
high_alpha = highcut_alpha / nyq
b_alpha, a_alpha = butter(order, [low_alpha, high_alpha], btype='band')
# ALPHA + BETA
lowcut = 8
highcut = 30
low = lowcut / nyq
high = highcut / nyq
b_alp_be, a_alp_be = butter(order, [low, high], btype='band')

## CAR Options

In [ ]:
#Copied from Weichwald repository.
# returns basis of A's null space
def null(A, eps=1e-15):
    # svd
    u, s, v = np.linalg.svd(A)
    # dimension of null space
    padding = max(0, np.shape(A)[1] - np.shape(s)[0])
    # select columns/rows corresponding to v
    null_mask = np.concatenate(((s <= eps),
                                np.ones((padding,), dtype=bool)), axis=0)
    null_space = np.compress(null_mask, v, axis=0)
    return null_space


def carcomplement(samples):
    d = samples.shape[0]
    carcomp = null(np.ones((1, d)))
    return carcomp.dot(samples)

## Function for loading data and converting to correct form

In [ ]:
#Loading data functions
NO_channels = 22
def extract_data(subject,training,PATH):
    # Arguments:
    # Subject, number 1 to 9
    # Training or evaluation, boolean value
    # Absolute path
    # Return: Tuple where
    # data_rutern: numpy matrix, size = NO_valid_trial x 22 x window_length
    # class_return: numpy matrix, size = NO_valid_trial  
    
    NO_channels = 22
    NO_tests = 6*48
    Window_Length = 3*250
    
    data_return = np.zeros((NO_tests,NO_channels,Window_Length))
    class_return = np.zeros(NO_tests)
    #subject_return = np.zeros(NO_tests)
    
    NO_valid_trial = 0
    if training:
        a = sio.loadmat(PATH+'A0'+str(subject)+'T.mat')
    else:
        a = sio.loadmat(PATH+'A0'+str(subject)+'E.mat')
        
    a_data = a['data']
    for i in range(0,a_data.size):
        a_data1     = a_data[0,i]
        a_data2     = [a_data1[0,0]]
        a_data3     = a_data2[0]
        a_X         = a_data3[0]
        a_trial     = a_data3[1]
        a_y         = a_data3[2]
        a_artifacts = a_data3[5]
        for trial in range(0,a_trial.size):
            if(a_artifacts[trial]==0):  
                data_return[NO_valid_trial,:,:] = np.transpose(a_X[int(a_trial[trial])+750:(int(a_trial[trial])+Window_Length)+750,:22])
                class_return[NO_valid_trial] = int(a_y[trial])
                #subject_return[NO_valid_trial]= subject
                NO_valid_trial += 1
    return data_return[0:NO_valid_trial,:,:], class_return[0:NO_valid_trial]
path = "/Users/Svesketerning/Google-Drev/Project in Statistics/data/"

def append_pers_data(pers_list):
    
    eeg_data = list(extract_data(pers_list[0],True,path))
    eeg_temp_init = list(extract_data(pers_list[0],False,path)) # Doing it like this because I am dumb
    eeg_data[0] = np.append(eeg_data[0],eeg_temp_init[0], axis = 0)
    eeg_data[1] = np.append(eeg_data[1],eeg_temp_init[1], axis = 0)
    for i in pers_list[1:]:
        eeg_temp = list(extract_data(i,True,path))
        eeg_data[0] = np.append(eeg_data[0],eeg_temp[0], axis = 0)
        eeg_data[1] = np.append(eeg_data[1],eeg_temp[1], axis = 0)
        eeg_temp2 = list(extract_data(i,False,path))
        eeg_data[0] = np.append(eeg_data[0],eeg_temp2[0], axis = 0)
        eeg_data[1] = np.append(eeg_data[1],eeg_temp2[1], axis = 0)
    return eeg_data

## Making subsets to pass into pipeline and defining CAR function

In [ ]:
# Making subsets to pass into pipeline
def subset_make(no_test):
    subset_choices = []
    for i in list(range(1,9)): # People to test on 1-9 range(1,9)
        prior_choices = []
        for ii in range(no_test): # Number of tests
            choice_persons = list(np.random.choice(range(1,10),i,replace=False)) #Training subset
            #while choice_persons in prior_choices:
            #    choice_persons = list(np.random.choice(range(1,10),i,replace=False))
            prior_choices.append(choice_persons)
        subset_choices.append(prior_choices)
    return subset_choices

In [ ]:
def CAR(X,Y,car):
    if car:
        X,Y = carcomplement(X),carcomplement(Y)
    return X,Y

## Amateur built pipeline

In [ ]:
def ICA_pipeline(no_people,no_test,ICA_list,classifier,car,alpha):
    # no_people; List of number of people to test on, e.g. [3,5,8]
    # no_test; Number of test for each number of people, max 8 
    # ICA_list; List of ICA method to test
    #classifier; which classifier to use
    #CAR; Should the function perform CAR, True or False
    #alpha; Should you use both a alpha and beta filter or just beta filter, True or False
    subset_choices = subset_make(no_test)
    accuracy = []
    for i in tqdm(no_people): 
        for ii in tqdm(range(no_test)):
            current_subset = (subset_choices[i-1])[ii] # Subsetting 
            X1,Y1 = append_pers_data(current_subset),append_pers_data(list(set(range(1,10))-set(current_subset)))
            X,Y = np.concatenate([X1[0][c] for c in range(len(X1[0]))],axis = 1),np.concatenate([Y1[0][c] for c in range(len(Y1[0]))],axis = 1)
            X,Y = CAR(X,Y,car)
            X,Y = X.transpose(),Y.transpose()
            # ICA
            for method in ICA_list:
                transformer = method
                X_transform = (transformer.fit_transform(X)).transpose()
                if alpha:
                    X_transform_alpha = lfilter(b_alpha,a_alpha,X_transform)
                    X_transform_alpha = X_transform_alpha.reshape((NO_channels-1*car, len(X1[0]), 750))
                    # Features if alpha
                    Features_alpha = np.log(np.var(X_transform_alpha, axis = 2))
                    Features_alpha = Features_alpha.transpose()
                X_transform = lfilter(b_beta,a_beta,X_transform) #Insert a,b _alp_bet + alpha=False if one wants wide band range                              
                X_transform = X_transform.reshape((NO_channels-1*car, len(X1[0]), 750))
                
                # Features
                Features = np.log(np.var(X_transform, axis = 2))
                Features = Features.transpose()
                
                if alpha:
                    Features = np.column_stack((Features,Features_alpha))
                # Classifier
                clf = classifier
                clf = clf.fit(Features, X1[1])
                
                # Evaluation 
                Y_transform = (transformer.transform(Y)).transpose()
                
                if alpha:
                    Y_transform_alpha = lfilter(b_alpha,a_alpha,Y_transform)
                    Y_transform_alpha = Y_transform_alpha.reshape((NO_channels-1*car, len(Y1[0]), 750))
                    # Features is alpha
                    Features_Y_alpha = np.log(np.var(Y_transform_alpha, axis = 2))
                    Features_Y_alpha = Features_Y_alpha.transpose()
                        
                Y_transform = lfilter(b_beta,a_beta,Y_transform)
                Y_transform = Y_transform.reshape((NO_channels-1*car, len(Y1[0]), 750))
                Features_Y = np.log(np.var(Y_transform, axis = 2))
                Features_Y = Features_Y.transpose()
                
                if alpha:
                    Features_Y = np.column_stack((Features_Y,Features_Y_alpha))     
                accuracy.append(np.mean(clf.predict(Features_Y)==Y1[1]))
    # Accuracy list order: method 1 * subset 1,method 2 * subset 1... method n * subset1, method 1 * subset 2...
    return accuracy

##  Parameter choices, ICA's and Classifiers

In [ ]:
eps = 10**(-12)
n_iter_max = 10000 
n_components = NO_channels-1 # Number of components to extract, minus 1 because of CAR
partition_size = 3000 # Same as Pfister, Weichwald et al. 2019.
partition_sizes = 10**10 # Greater than number of observations >30*10^6.
group_size = 520*750 # Approximately one for each trial

In [ ]:
# w/ SOBI
ica_method_list = [FastICA(tol=eps,max_iter=n_iter_max), # fastICA
                CoroICA(#coroICA (var)
                    partitionsize = partition_size,
                    groupsize = int(500*750), #group_size
                    tol=eps,
                    max_iter=n_iter_max,
                    condition_threshold=1000,
                    pairing='neighbouring',
                    instantcov=True,
                    timelags=None),
                UwedgeICA(#choiICA (var)
                    partitionsize = partition_size,
                    tol=eps,
                    max_iter=n_iter_max,
                    condition_threshold=1000,
                    instantcov=True,
                    timelags=None),
                UwedgeICA(partitionsize=partition_sizes, #SOBI
                    tol=eps,
                    max_iter=n_iter_max,
                    condition_threshold=1000,
                    instantcov=False, # False in SOBI
                    timelags=list(range(1, 14)))]

In [ ]:
# w/o SOBI
ica_method_list2 = [FastICA(tol=eps,max_iter=n_iter_max), # fastICA
                CoroICA(#coroICA (var)
                    partitionsize = partition_size,
                    groupsize = int(500*750), #group_size
                    tol=eps,
                    max_iter=n_iter_max,
                    condition_threshold=1000,
                    pairing='neighbouring',
                    instantcov=True,
                    timelags=None),
                UwedgeICA(#choiICA (var)
                    partitionsize = partition_size,
                    tol=eps,
                    max_iter=n_iter_max,
                    condition_threshold=1000,
                    instantcov=True,
                    timelags=None)]

In [ ]:
# Classifiers
quad_clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0.1,
               store_covariance=False, store_covariances=None, tol=0.0001)
rf_clf = RandomForestClassifier(max_depth=2, random_state=0)

## Pipeline study

In [ ]:
##############################################
############### Pipeline study ###############
##############################################
# Quad_clf, CAR, alpha+beta
accuracy_list1 = ICA_pipeline([4],20,ica_method_list,quad_clf,True,True)

# Quad_clf, CAR, beta
accuracy_list2 = ICA_pipeline([4],20,ica_method_list,quad_clf,True,False)

# Quad_clf, no-CAR, alpha+beta
accuracy_list3 = ICA_pipeline([4],20,ica_method_list,quad_clf,False,False)

# Quad_clf, no-CAR, beta
accuracy_list4 = ICA_pipeline([4],20,ica_method_list,quad_clf,False,False)

# rf_clf, CAR, alpha+beta
accuracy_list5 = ICA_pipeline([4],20,ica_method_list,rf_clf,True,True)

#rf_clf, CAR, beta
accuracy_list6 = ICA_pipeline([4],20,ica_method_list,rf_clf,True,False)

# rf_clf, no-CAR, alpha+beta
accuracy_list7 = ICA_pipeline([4],20,ica_method_list,rf_clf,False,True)

# rf_clf, no-CAR, beta
accuracy_list8 = ICA_pipeline([4],20,ica_method_list,rf_clf,False,False)

In [ ]:
collected = [] # Collect data
collected.extend(accuracy_list1)
collected.extend(accuracy_list2)
collected.extend(accuracy_list3)
collected.extend(accuracy_list4)
collected.extend(accuracy_list5)
collected.extend(accuracy_list6)
collected.extend(accuracy_list7)
collected.extend(accuracy_list8)

## Function to make accuracy score to dataframe (Note the order the data is stored in in pipeline function)
def main1(accuracy_list):
    df = pd.DataFrame(columns = ['ICA_method','Pipeline','Accuracy'])
    df = df.assign(Accuracy = accuracy_list)
    df = df.assign(Train_Size = np.repeat(4,20*4*8))
    df = df.assign(Pipeline = np.repeat(['QDA & CAR & Alpha+Beta',
                                        'QDA & CAR & Beta',
                                        'QDA & no-CAR & Alpha+Beta',
                                        'QDA & no-CAR & Beta', 
                                        'RF & CAR & Alpha+Beta',
                                        'RF & CAR & Beta',
                                        'RF & no-CAR & Alpha+Beta',
                                        'RF & no-CAR & Beta'],20*4))
    df = df.assign(ICA_method = ['fastICA','coroICA (var)','choiICA (var)', 'SOBI']*20*8)
    return df
df = main1(collected)

### Plot Creation 

In [ ]:
sns.set(rc={'figure.figsize':(30,20)})
sns.set(font_scale = 2)
sns.set_style("whitegrid")
fig, axes = plt.subplots(2, 1, sharex=False,sharey = True)
sns.despine(left=True)

bx = sns.boxplot(x="Pipeline", y="Accuracy", hue="ICA_method", 
                    palette="Blues_d",saturation=0.65, 
                    fliersize = 0, whis = 1.27,
                     data=df[:320], ax=axes[0])
cx = sns.boxplot(x="Pipeline", y="Accuracy", hue="ICA_method", 
                    palette="Blues_d",saturation=0.65, 
                    fliersize = 0, whis = 1.27,
                     data=df[320:], ax=axes[1])
bx.axhline(0.25, xmin = -1, xmax=8, color = 'r', linewidth = 4, linestyle = 'dashed')
cx.axhline(0.25, xmin = -1, xmax=8, color = 'r', linewidth = 4, linestyle = 'dashed')
plt.legend(fontsize='medium', loc='upper right')
cx.legend().remove()
fig.tight_layout()

### Saving data

In [ ]:
fig = bx.get_figure()
fig.savefig('pipeline-figure.pdf', format='pdf', dpi=1200)
df.to_csv('pipeline-study.csv')

## Comparison to wide filter (should change pipeline function before running)

In [ ]:
accuracy_list9 = ICA_pipeline([4],20,ica_method_list2,quad_clf,True,False)
accuracy_list10 = ICA_pipeline([4],20,ica_method_list2,quad_clf,False,False)

In [ ]:
collected2 = []
collected2.extend(accuracy_list9)
collected2.extend(accuracy_list10)
def main2(accuracy_list):
    df = pd.DataFrame(columns = ['ICA_method','Pipeline','Accuracy'])
    df = df.assign(Accuracy = accuracy_list)
    df = df.assign(Train_Size = np.repeat(4,20*3*2))
    df = df.assign(Pipeline = np.repeat(['QDA & CAR & alpha/beta',
                                        'QDA & no-CAR & alpha/beta'],20*3*8))
    df = df.assign(ICA_method = ['fastICA','coroICA (var)','choiICA (var)']*20*2)
    return df
df2 = main2(collected2)
df3 = pd.DataFrame.append(df,df2) # Collect the big df with the alpha/beta for comparison
##### delete unnecessary data (doing like this because I could not figure out a better way) #####
df3.drop(df.loc[df['ICA_method'] == 'SOBI'].index, inplace=True)
df3.drop(df.loc[df['Pipeline'] == 'RF & CAR & Alpha+Beta'].index, inplace=True)
df3.drop(df.loc[df['Pipeline'] == 'RF & CAR & Beta'].index, inplace=True)
df3.drop(df.loc[df['Pipeline'] == 'RF & no-CAR & Alpha+Beta'].index, inplace=True)
df3.drop(df.loc[df['Pipeline'] == 'RF & no-CAR & Beta'].index, inplace=True)
df3.drop(df.loc[df['Pipeline'] == 'QDA & CAR & Beta'].index, inplace=True)
df3.drop(df.loc[df['Pipeline'] == 'QDA & no-CAR & Beta'].index, inplace=True)

### Plot Creation

In [ ]:
sns.set(rc={'figure.figsize':(30,10)})
sns.set(font_scale = 2)
sns.set_style("whitegrid")
fig, axes = plt.subplots(nrows = 1, ncols = 1)
bx = sns.boxplot(x="Pipeline", y="Accuracy", hue="ICA_method", 
                    palette="Blues_d",saturation=0.65, 
                    fliersize = 0, whis = 1.27,
                     data=df3)
bx.axhline(0.25, xmin = -1, xmax=5, color = 'r', linewidth = 4, linestyle = 'dashed')
plt.legend(fontsize='medium', loc='upper right')
fig.tight_layout()

### Saving data

In [ ]:
fig = bx.get_figure()
fig.savefig('bands-comparison.pdf', format='pdf', dpi=1200)
df3.to_csv('bands-comparison.csv')

## Different size subset analysis

In [ ]:
accuracy_list11 = ICA_pipeline([1,2,3,4,5,6,7,8],20,ica_method_list2,quad_clf,False,False)

In [ ]:
def main3(accuracy_list):
    df = pd.DataFrame(columns = ['ICA_method','Pipeline','Accuracy'])
    df = df.assign(Accuracy = accuracy_list)
    df = df.assign(Train_Size = np.repeat([1,2,3,4,5,6,7,8],20*3))
    df = df.assign(Pipeline = np.repeat(['QDA & CAR & Beta'],20*8*3))
    df = df.assign(ICA_method = ['fastICA','coroICA (var)','choiICA (var)']*20*8)
    return df
df4 = main3(accuracy_list11)

### Plot creation (lineplot)

In [ ]:
sns.set(rc={'figure.figsize':(25,10)})
sns.set(font_scale = 2)
sns.set_style("whitegrid")
fig, axes = plt.subplots(nrows = 1, ncols = 1)
bx = sns.lineplot(x="Train_Size", y="Accuracy", hue="ICA_method", 
                    palette= "colorblind",
                     data=df4)
bx.axhline(0.25, xmin = -1, xmax= 1, color = 'r', linewidth = 2, linestyle = 'dashed')
plt.legend(fontsize='small', loc='upper left')
fig.tight_layout()

### Saving data 

In [ ]:
fig = bx.get_figure()
fig.savefig('big-analysis.pdf', format='pdf', dpi=1200)
df4.to_csv('big-analysis.csv')

### END 